<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/Document_retrieval_with_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install langchain
!pip install faiss-gpu
!pip install langchain_community
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5

In [2]:
from datasets import load_dataset
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

dataset = load_dataset("lamini/product-catalog-questions", split='train')
print(dataset[0])

documents = []
for row in dataset:
  documents.append(
      Document(
          page_content=row['description'],
          metadata={'product_name': row['product_name']}
      )
  )

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
processed_documents = text_splitter.split_documents(documents)
# print(processed_documents[0])

# embedder = HuggingFaceEmbeddings(
#     model_name="thuan9889/llama_embedding_model_v1",
#     encode_kwargs={'normalize_embeddings': False},
# )
embedder = HuggingFaceEmbeddings(
    model_name="thuan9889/llama_embedding_model_v1"
)

vector_store = FAISS.from_documents(processed_documents, embedder)

query = "What type of organic chicken is available?"
docs = vector_store.similarity_search(query)
print(docs[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/27443 [00:00<?, ? examples/s]

{'product_id': '9517', 'product_name': 'Havarti Cheese', 'aisle_id': '21', 'department_id': '16', 'question': 'What type of cheese would be a good addition to a charcuterie board with apples or pears ', 'description': 'Havarti cheese is a type of semi-soft, creamy cheese that originates from the Caucasus region of Georgia.  It is made from the milk of goats or sheep, and is known for its rich, nutty flavor and smooth texture.  Unlike other cheeses, Havarti cheese is not aged for a long time, resulting in a fresher, more delicate taste'}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

page_content='Introducing our premium organic chicken breast, raised without antibiotics or hormones.  It's a lean and healthy protein source, perfect for any meal.  Our chicken is sourced from small-scale farms that prioritize animal welfare and environmental sustainability' metadata={'product_name': 'Organic Chicken Breast'}


In [3]:
retriever = vector_store.as_retriever()

system_prompt = """You are an assistant that answers customer service questions.
Use the following retrieved context to answer the question.
If you don't know the answer, say 'I don't know.'.
Answer in a single sentence.

Context: {context}

Question: {input}

Answer: """

# prompt = PromptTemplate(input_variables=["context", "question"], template=system_prompt)
prompt = PromptTemplate.from_template(system_prompt)


In [4]:
llm = HuggingFacePipeline.from_model_id(
    model_id="nvidia/Llama3-ChatQA-1.5-8B",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 50})

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)
generator_chain = (
    RunnablePassthrough
    .assign(input=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

retrieve_docs = (lambda x: x['input']) | retriever
rag_chain = RunnablePassthrough.assign(context=retrieve_docs).assign(answer=generator_chain)
response = rag_chain.invoke({"input": "How are your chicken products raised?"})
print(response)

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

{'input': 'How are your chicken products raised?', 'context': [Document(metadata={'product_name': 'Fresh Whole Chicken'}, page_content='Introducing our fresh whole chicken, a premium product that is perfect for your next meal.  Our chicken is raised on a diet of natural grains and is free of antibiotics and hormones.  It is a versatile ingredient that can be used in a variety of dishes, from roasting to grilling to baking'), Document(metadata={'product_name': 'Fresh Whole Chicken'}, page_content='Introducing our fresh whole chicken, a premium product that is perfect for your next meal.  Our chicken is raised on a diet of natural grains and is free of antibiotics and hormones.  It is a versatile ingredient that can be used in a variety of dishes, from roasting to grilling to baking'), Document(metadata={'product_name': 'Fresh Whole Chicken'}, page_content='Introducing our fresh whole chicken, a premium product that is perfect for your next meal.  Our chicken is raised on a diet of natur